# Twitter Accounts to Follow

@OpenANC should generally follow all current commissioners and candidates.

In [1]:
import os
os.chdir('../')

In [2]:
import time
import tweepy
import random
import pandas as pd

from scripts.data_transformations import list_commissioners

consumer_key = os.environ.get('TWITTER_API_KEY')
consumer_secret = os.environ.get('TWITTER_API_KEY_SECRET')
access_key = os.environ.get('TWITTER_ACCESS_TOKEN')
access_secret = os.environ.get('TWITTER_ACCESS_TOKEN_SECRET')

In [3]:
people = pd.read_csv('data/people.csv')
candidates = pd.read_csv('data/candidates.csv')
candidate_statuses = pd.read_csv('data/candidate_statuses.csv')
commissioners = list_commissioners(status='current')

people_candidates = pd.merge(people, candidates, how='inner', on='person_id')
people_candidates_status = pd.merge(people_candidates, candidate_statuses, how='inner', on='candidate_status')
people_candidates_active = people_candidates_status[
    (people_candidates_status['count_as_candidate'])
    & (people_candidates_status['election_year'] == 2022)
].copy()

people_commissioners = pd.merge(people, commissioners, how='inner', on='person_id')

# Group all twitter links for active candidates and current commissioners in one list
twitter_links = pd.concat([people_commissioners.twitter_link, people_candidates_active.twitter_link])

In [4]:
twitter_handles = (
    twitter_links[twitter_links.notnull()]
    .str.replace('https://twitter.com/', '')
    .str.lower()
)

list_of_handles = sorted(twitter_handles.unique())

In [5]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth)

In [6]:
openanc_follows = []
for user in tweepy.Cursor(api.friends, screen_name='OpenANC').items():
    openanc_follows += [user.screen_name.lower()]
    time.sleep(1)

In [7]:
should_follow = [t for t in list_of_handles if t not in openanc_follows]
len(should_follow)

40

In [10]:
# # Pick 5 at random
# follow_me = random.sample(should_follow, 5)

# for f in follow_me:
#     print('https://twitter.com/' + f)

In [9]:
for f in should_follow:
    print('https://twitter.com/' + f)

https://twitter.com/laroya_ward4
https://twitter.com/latestchristian
https://twitter.com/laurenrogersdc
https://twitter.com/leftofdacenter
https://twitter.com/lucrsaintgenies
https://twitter.com/matt_brandeburg
https://twitter.com/mattlafortune
https://twitter.com/maxdevilliers
https://twitter.com/michael_sriqui
https://twitter.com/michaelzeldin
https://twitter.com/mikeindeecee1
https://twitter.com/milleranc1a05
https://twitter.com/mshankle
https://twitter.com/msmansaray
https://twitter.com/mwraydc
https://twitter.com/oblivious_dude
https://twitter.com/petergosselin
https://twitter.com/pritapiekara
https://twitter.com/qcolonroosevelt
https://twitter.com/raamin5b04
https://twitter.com/randydaledowns
https://twitter.com/rashidabrowndc
https://twitter.com/ready_4_dc
https://twitter.com/reneelbowser
https://twitter.com/revwendy3
https://twitter.com/sabelharris
https://twitter.com/santi4dc
https://twitter.com/sherenejoseph
https://twitter.com/soundpolicy
https://twitter.com/teamalgward7
htt